In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
import os
import re
from scipy.signal import butter, filtfilt, argrelextrema
from scipy import interpolate


##### Gathering data


In [ ]:
# Control data
fmap_dir_control = ./Control
fmap_data_control = glob(os.path.join(fmap_dir_control, '**/SittingDown_Chair_FMA0[0-9]_C[0-9]*.tsv'), recursive=True)

#PwP data
fmap_dir_pwp = ./PWP
fmap_data_pwp = glob(os.path.join(fmap_dir_pwp, '**/SittingDown_Chair_FMA0[0-9]_P[0-9]*.tsv'), recursive=True)
fmap_data_pwp


### Data passes thorugh low-pass butterworth filter of 4th order


In [ ]:

def butter_lowpass_filter(data, cutoff_frequency=10, sampling_rate=100, order=4):
    nyquist = 0.5 * sampling_rate
    normal_cutoff = cutoff_frequency / nyquist
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    filtered_data = filtfilt(b, a, data, axis=0, padlen=0)
    return filtered_data




## Reading file function


In [ ]:

NAN_special = -999999
def read_file(file):
    # Load the motion capture data into a Pandas DataFrame
    df = pd.read_csv(file,sep = '\t',skiprows=11)
    df = df.fillna(NAN_special)
    df.columns = df.columns.str.strip() #eliminating unecesary spaces in column titles
    time =  df.loc[:, "Time"].to_numpy()
    
   
    marker_list = ['RSHO X', 'RSHO Y', 'RSHO Z', 'LSHO X', 'LSHO Y','LSHO Z', 
                   'LFHD X', 'LFHD Y', 'LFHD Z', 'RFHD X', 'RFHD Y', 'RFHD Z',
                   'STRN X', 'STRN Y', 'STRN Z',
                   'LASI X', 'LASI Y', 'LASI Z', 'RASI X', 'RASI Y', 'RASI Z',
                   'LTOE X', 'LTOE Y', 'LTOE Z','RTOE X', 'RTOE Y', 'RTOE Z',
                  'LANK X', 'LANK Y', 'LANK Z','RANK X', 'RANK Y', 'RANK Z',
                  'LHEE X', 'LHEE Y', 'LHEE Z','RHEE X', 'RHEE Y', 'RHEE Z',
                  'LKNE X', 'LKNE Y', 'LKNE Z','RKNE X', 'RKNE Y', 'RKNE Z']

    # create empty arrays for each marker
    RSHO = np.zeros((len(time), 3))
    LSHO = np.zeros((len(time), 3))
    LSTC = np.zeros((len(time), 3))
    RBAK = np.zeros((len(time), 3))
    LFHD = np.zeros((len(time), 3))
    RFHD = np.zeros((len(time), 3))
    LBHD = np.zeros((len(time), 3))
    RBHD = np.zeros((len(time), 3))
    STRN = np.zeros((len(time), 3))
    LASI = np.zeros((len(time), 3))
    RASI = np.zeros((len(time), 3))
    LPSI = np.zeros((len(time), 3))
    RPSI = np.zeros((len(time), 3))
    RTOE = np.zeros((len(time), 3))
    LTOE = np.zeros((len(time), 3))
    LANK = np.zeros((len(time), 3))
    RANK = np.zeros((len(time), 3))
    LHEE = np.zeros((len(time), 3))
    RHEE = np.zeros((len(time), 3))
    LKNE = np.zeros((len(time), 3))
    RKNE = np.zeros((len(time), 3))
    
    flag = False # this flAg will help us identify the files that are incomplete later 
    missing_markers = [] # this is a list to collect the missing markers in the file
    
    for col_name in marker_list:
        if col_name in df.columns:
            
            if col_name.startswith('RSHO'):
                RSHO = df.loc[:, ['RSHO X', 'RSHO Y', 'RSHO Z']].to_numpy()
                RSHO = butter_lowpass_filter(RSHO)
            elif col_name.startswith('LSHO'):
                LSHO = df.loc[:, ['LSHO X', 'LSHO Y', 'LSHO Z']].to_numpy()
                LSHO = butter_lowpass_filter(LSHO)
            elif col_name.startswith('LFHD'):
                LFHD = df.loc[:, ['LFHD X', 'LFHD Y', 'LFHD Z']].to_numpy()
                LFHD = butter_lowpass_filter(LFHD)
            elif col_name.startswith('RFHD'):
                RFHD = df.loc[:, ['RFHD X', 'RFHD Y', 'RFHD Z']].to_numpy()
                RFHD = butter_lowpass_filter(RFHD)
            elif col_name.startswith('STRN'):
                STRN = df.loc[:, ['STRN X', 'STRN Y', 'STRN Z']].to_numpy()
                STRN = butter_lowpass_filter(STRN)
            elif col_name.startswith('LASI'):
                LASI = df.loc[:, ['LASI X', 'LASI Y', 'LASI Z']].to_numpy()
                LASI = butter_lowpass_filter(LASI)
            elif col_name.startswith('RASI'):
                RASI = df.loc[:, ['RASI X', 'RASI Y', 'RASI Z']].to_numpy()
                RASI = butter_lowpass_filter(RASI)
            elif col_name.startswith('RTOE'):
                RTOE = df.loc[:, ['RTOE X', 'RTOE Y', 'RTOE Z']].to_numpy()
                RTOE = butter_lowpass_filter(RTOE)
            elif col_name.startswith('LTOE'):
                LTOE = df.loc[:, ['LTOE X', 'LTOE Y', 'LTOE Z']].to_numpy()
                LTOE = butter_lowpass_filter(LTOE)
            elif col_name.startswith('LANK'):
                LANK = df.loc[:, ['LANK X', 'LANK Y', 'LANK Z']].to_numpy()
                LANK = butter_lowpass_filter(LANK)
            elif col_name.startswith('RANK'):
                RANK = df.loc[:, ['RANK X', 'RANK Y', 'RANK Z']].to_numpy()
                RANK = butter_lowpass_filter(RANK)
            elif col_name.startswith('LHEE'):
                LHEE = df.loc[:, ['LHEE X', 'LHEE Y', 'LHEE Z']].to_numpy()
                LHEE = butter_lowpass_filter(LHEE)
            elif col_name.startswith('RHEE'):
                RHEE = df.loc[:, ['RHEE X', 'RHEE Y', 'RHEE Z']].to_numpy()
                RHEE = butter_lowpass_filter(RHEE)
            elif col_name.startswith('LKNE'):
                LKNE = df.loc[:, ['LKNE X', 'LKNE Y', 'LKNE Z']].to_numpy()
                LKNE = butter_lowpass_filter(LKNE)
            elif col_name.startswith('RKNE'):
                RKNE = df.loc[:, ['RKNE X', 'RKNE Y', 'RKNE Z']].to_numpy()
                RKNE = butter_lowpass_filter(RKNE)
        else:
            flag = True
            missing_markers.append(col_name)
#             print('File: {} , does not have marker {}'.format(file, col_name))
    print(file,missing_markers)
    return flag,missing_markers,time,LFHD,RFHD,LBHD,RBHD,RSHO,LSHO,LSTC,RBAK,STRN,LASI,RASI,LPSI,RPSI,RTOE,LTOE,LANK,RANK,LHEE,RHEE,LKNE,RKNE


In [ ]:
def interpolation(marker, t):
    ORIGINAL_FS = 1 / (t[1] - t[0])

    # Concatenate 3D positions to a matrix of size Nsamples x 3
    marker_3Dpos = np.c_[marker[:, 0], marker[:, 1], marker[:, 2]]

    # Compute 3D velocity
    marker_3Dvel = np.diff(marker_3Dpos, axis=0) / np.diff(t)[:, None]

    # Compute tangential velocity
    marker_velAbs = np.sqrt(np.sum(marker_3Dvel ** 2, axis=1))

    # Compute 3D acceleration
    marker_3Dacc = np.diff(marker_3Dvel, axis=0) / np.diff(t[:-1])[:, None]

    # Compute tangential acceleration
    marker_accAbs = np.sqrt(np.sum(marker_3Dacc ** 2, axis=1))

    # Look for outliers in the tangential acceleration signal
    acc_thresh = 10000  # mm/s^2 threshold above which we consider an outlier
    ind_outliers = np.squeeze(np.argwhere(marker_3Dpos[:, 0] == NAN_special))
    DELTA_second = 1 / ORIGINAL_FS
    DELTA_sample = int(np.round(DELTA_second * ORIGINAL_FS))
    for delta in np.arange(-DELTA_sample, DELTA_sample, 1):
        ind_outliers = np.union1d(ind_outliers, np.squeeze(np.argwhere(marker_accAbs > acc_thresh)) + 2 + delta)  # add 2 because of use of first difference 2 times
    ind_outliers = np.intersect1d(np.arange(1, len(t)), ind_outliers)

    # Interpolate if necessary
    marker_3Dpos_interp = copy.deepcopy(marker_3Dpos)
    if len(ind_outliers) > 0:
        ind_notoutliers = np.setdiff1d(np.arange(len(t)), ind_outliers)
        marker_3Dpos_interp = interpolate.pchip_interpolate(t[ind_notoutliers], marker_3Dpos[ind_notoutliers, :], t, axis=0)

    return marker_3Dpos_interp

In [ ]:
flag,missing_markers,time,LFHD,RFHD,LBHD,RBHD,RSHO,LSHO,LSTC,RBAK,STRN,LASI,RASI,LPSI,RPSI,RTOE,LTOE,LANK,RANK,LHEE,RHEE,LKNE,RKNE = read_file(fmap_data_pwp[1])

## Peak AP  inclination angle (yz plane)


In [ ]:
def calculate_peak_AP_inclination(left_shoulder,right_shoulder,sternum, missing_markers_list):
    """
    Calculate the peak anterior-posterior (AP) inclination of the trunk based on the position of shoulder and sternum markers.

    Parameters:
    - left_shoulder (numpy.ndarray): Position vector representing the left shoulder marker.
    - right_shoulder (numpy.ndarray): Position vector representing the right shoulder marker.
    - sternum (numpy.ndarray): Position vector representing the sternum marker.

    Returns:
    - float: The peak AP inclination angle in degrees.
    """
    # Normalize positions by subtracting initial positions
#     left_shoulder_rel = left_shoulder - left_shoulder[0]
#     right_shoulder_rel = right_shoulder - right_shoulder[0]
#     sternum_rel = sternum - sternum[0]
    if any(marker.startswith(('LSHO', 'RSHO', 'STRN')) for marker in missing_markers_list):
        if all(marker.startswith(('LSHO', 'RSHO', 'STRN')) for marker in missing_markers_list):
            print('All required markers are missing, returning NaN')
            return np.nan, np.zeros_like(time)
        else:
            print('One or more required markers are missing, calculating trunk position with available markers')
            available_markers = []
            if not any(marker.startswith('LSHO') for marker in missing_markers_list):
                available_markers.append(left_shoulder)
            if not any(marker.startswith('RSHO') for marker in missing_markers_list):
                available_markers.append(right_shoulder)
            if not any(marker.startswith('STRN') for marker in missing_markers_list):
                available_markers.append(sternum)
            trunk = np.mean(available_markers, axis=0)
    else:
        # Calculate the trunk position based on relative positions
        trunk = (left_shoulder + right_shoulder + sternum) / 3
    
    # Calculate the inclination angle with respect to the yz plane for each vector
    inclination_angle = np.arctan2(trunk[:, 2], trunk[:, 1])

    inclination_angle_degrees = 180-np.degrees(inclination_angle)


    inclination_angle_degrees = inclination_angle_degrees
    peak_AP_inclination = max(inclination_angle_degrees)
    
    
    return peak_AP_inclination, inclination_angle_degrees




## Peak ML inclination angle (xz plane)

The peak ML inclination is calculated with the xz coordiantes of the sternum marker. The max inclination represents the max deviation of the sternum in the frontal plane from its initial position at the start of the turn. 



In [ ]:
def calculate_peak_ML_inclination(left_shoulder,right_shoulder,sternum, missing_markers_list):
    """
    Calculate the peak medial-lateral (ML) inclination of the trunk based on the position of shoulder and sternum markers.

    Parameters:
    - left_shoulder (numpy.ndarray): Position vector representing the left shoulder marker.
    - right_shoulder (numpy.ndarray): Position vector representing the right shoulder marker.
    - sternum (numpy.ndarray): Position vector representing the sternum marker.

    Returns:
    - float: The peak ML inclination angle in degrees.
    """
 
    if any(marker.startswith(('LSHO', 'RSHO', 'STRN')) for marker in missing_markers_list):
        if all(marker.startswith(('LSHO', 'RSHO', 'STRN')) for marker in missing_markers_list):
            print('All required markers are missing, returning NaN')
            return np.nan, np.zeros_like(time)
        else:
            print('One or more required markers are missing, calculating trunk position with available markers')
            available_markers = []
            if not any(marker.startswith('LSHO') for marker in missing_markers_list):
                available_markers.append(left_shoulder)
            if not any(marker.startswith('RSHO') for marker in missing_markers_list):
                available_markers.append(right_shoulder)
            if not any(marker.startswith('STRN') for marker in missing_markers_list):
                available_markers.append(sternum)
            trunk = np.mean(available_markers, axis=0)
    else:
        # Calculate the trunk position based on relative positions
        trunk = (left_shoulder + right_shoulder + sternum) / 3
    


        # Calculate the inclination angle along the xz plane for each vector
    inclination_angle = np.arctan2(trunk[:, 2], trunk[:, 0])


    inclination_angle_degrees = np.degrees(inclination_angle)
    inclination_angle_degrees -= inclination_angle_degrees[0]
    peak_ML_inclination = max(np.abs(inclination_angle_degrees))
   
    return peak_ML_inclination ,inclination_angle_degrees

In [ ]:
def find_global_minima_and_inclination_angles(LSHO, RSHO, STRN,missing_markers_list):
    # Find the global minima in the z-axis of LSHO, RSHO, and STRN
    z_LSHO = LSHO[:, 2]
    z_RSHO = RSHO[:, 2]
    z_STRN = STRN[:, 2]

    # Find the minima for each point
    min_index_LSHO = np.argmin(z_LSHO)
    min_index_RSHO = np.argmin(z_RSHO)
    min_index_STRN = np.argmin(z_STRN)

    # Find the global minimum index among the three minima
    min_values = [z_LSHO[min_index_LSHO], z_RSHO[min_index_RSHO], z_STRN[min_index_STRN]]
    global_min_index = np.argmin(min_values)

    # Determine the corresponding index in the original arrays
    if global_min_index == 0:
        index = min_index_LSHO
    elif global_min_index == 1:
        index = min_index_RSHO
    else:
        index = min_index_STRN

    # Calculate the ML and AP inclination angles
    peak_ML_inclination, ml_inclination_angle_degrees = calculate_peak_ML_inclination(LSHO, RSHO, STRN,missing_markers_list)
    peak_AP_inclination, ap_inclination_angle_degrees = calculate_peak_AP_inclination(LSHO, RSHO, STRN,missing_markers_list)

    # Get the inclination angles at the global minima index
    ml_inclination_at_minima = ml_inclination_angle_degrees[index]
    ap_inclination_at_minima = ap_inclination_angle_degrees[index]

    return ml_inclination_at_minima, ap_inclination_at_minima, index
    # return ml_inclination_at_minima, ap_inclination_at_minima,ml_inclination_angle_degrees, ap_inclination_angle_degrees, index

ml_inclination_at_minima,ap_inclination_at_minima, global_min_index = find_global_minima_and_inclination_angles(LSHO, RSHO, STRN,missing_markers)




## RMS of trunk acceleration 

Calculation method from Palmieri et al. 2013


In [ ]:
def calculate_acc_rms(left_shoulder, right_shoulder, sternum, time, missing_markers_list):
    """
    Calculate the root mean square (RMS) of the trunk acceleration based on the position of shoulder and sternum markers.

    Parameters:
    - left_shoulder (numpy.ndarray): Position vector representing the left shoulder marker (N_samples x 3).
    - right_shoulder (numpy.ndarray): Position vector representing the right shoulder marker (N_samples x 3).
    - sternum (numpy.ndarray): Position vector representing the sternum marker (N_samples x 3).
    - time (numpy.ndarray): Time vector (N_samples).

    Returns:
    - float: The RMS of the trunk acceleration.
    - numpy.ndarray: Magnitudes of the trunk acceleration at each time step.
    """

    if any(marker.startswith(('LSHO', 'RSHO', 'STRN')) for marker in missing_markers_list):
        if all(marker.startswith(('LSHO', 'RSHO', 'STRN')) for marker in missing_markers_list):
            print('All required markers are missing, returning NaN')
            return np.nan
        else:
            print('One or more required markers are missing, calculating trunk position with available markers')
            available_markers = []
            if not any(marker.startswith('LSHO') for marker in missing_markers_list):
                left_shoulder_rel = left_shoulder - left_shoulder[0]
                available_markers.append(left_shoulder_rel)
            if not any(marker.startswith('RSHO') for marker in missing_markers_list):
                right_shoulder_rel = right_shoulder - right_shoulder[0]
                available_markers.append(right_shoulder_rel)
            if not any(marker.startswith('STRN') for marker in missing_markers_list):
                sternum_rel = sternum - sternum[0]
                available_markers.append(sternum_rel)
            trunk = np.mean(available_markers, axis=0)
    else:
        # Normalize positions by subtracting initial positions
        left_shoulder_rel = left_shoulder - left_shoulder[0]
        right_shoulder_rel = right_shoulder - right_shoulder[0]
        sternum_rel = sternum - sternum[0]
        # Calculate the trunk position based on relative positions
        trunk = (left_shoulder_rel + right_shoulder_rel + sternum_rel)/ 3

    # Calculate the trunk velocity
    trunk_3Dvel = (np.diff(trunk, axis=0) / np.diff(time)[:, np.newaxis] )/1000
    
    # Compute 3D acceleration
    
    trunk_3Dacc = np.diff(trunk_3Dvel, axis=0) / np.diff(time[1:])[:, np.newaxis] 

    # Calculate magnitude of acceleration
    mag_acc = np.linalg.norm(trunk_3Dacc, axis=1)
    
    # Calculate RMS of acceleration
    acc_rms = np.sqrt(np.mean(mag_acc**2))
    
    return acc_rms#, mag_acc  # Return RMS and magnitudes

acc_rms = calculate_acc_rms(LSHO,RSHO,STRN,time,missing_markers)
print(acc_rms)

## Mean trunk jerk


In [ ]:
def calculate_trunk_jerk(left_shoulder,right_shoulder,sternum,time, missing_markers_list):
    """
    Calculate the mean jerk of the trunk based on the position of shoulder and sternum markers.

    Parameters:
    - left_shoulder (numpy.ndarray): Position vector representing the left shoulder marker.
    - right_shoulder (numpy.ndarray): Position vector representing the right shoulder marker.
    - sternum (numpy.ndarray): Position vector representing the sternum marker.

    Returns:
    - float: The mean jerk of the trunk.
    """
    if any(marker.startswith(('LSHO', 'RSHO', 'STRN')) for marker in missing_markers_list):
        if all(marker.startswith(('LSHO', 'RSHO', 'STRN')) for marker in missing_markers_list):
            print('All required markers are missing, returning NaN')
            return np.nan
        else:
            print('One or more required markers are missing, calculating trunk position with available markers')
            available_markers = []
            if not any(marker.startswith('LSHO') for marker in missing_markers_list):
                left_shoulder_rel = left_shoulder - left_shoulder[0]
                available_markers.append(left_shoulder_rel)
            if not any(marker.startswith('RSHO') for marker in missing_markers_list):
                right_shoulder_rel = right_shoulder - right_shoulder[0]
                available_markers.append(right_shoulder_rel)
            if not any(marker.startswith('STRN') for marker in missing_markers_list):
                sternum_rel = sternum - sternum[0]
                available_markers.append(sternum_rel)
            trunk = np.mean(available_markers, axis=0)
    else:
        # Normalize positions by subtracting initial positions
        left_shoulder_rel = left_shoulder - left_shoulder[0]
        right_shoulder_rel = right_shoulder - right_shoulder[0]
        sternum_rel = sternum - sternum[0]
        # Calculate the trunk position based on relative positions
        trunk = (left_shoulder_rel + right_shoulder_rel + sternum_rel)/ 3


    # Calculate the trunk velocity
    trunk_3Dvel = (np.diff(trunk, axis=0) / np.diff(time)[:, np.newaxis]) /1000
    # Compute 3D acceleration
    trunk_3Dacc = np.diff(trunk_3Dvel, axis=0) / np.diff(time[1:])[:, np.newaxis]  
    # Compute 3D jerk 
    trunk_3Djerk = np.diff(trunk_3Dacc, axis=0) / np.diff(time[2:])[:, np.newaxis]  

    mean_trunk_jerk = np.mean(trunk_3Djerk)
    return mean_trunk_jerk#single value

mean_jerk = calculate_trunk_jerk(LSHO,RSHO,STRN,time,missing_markers)

mean_jerk



## Peak trunk velocity

In [ ]:
def calculate_3d_peak_velocity(left_shoulder,right_shoulder,sternum,time, missing_markers_list):
    """
    Calculate the peak velocity of the trunk based on the position of shoulder and sternum markers.

    Parameters:
    - left_shoulder (numpy.ndarray): Position vector representing the left shoulder marker.
    - right_shoulder (numpy.ndarray): Position vector representing the right shoulder marker.
    - sternum (numpy.ndarray): Position vector representing the sternum marker.

    Returns:
    - float: The peak velocity of the trunk in mm/s.
    """
    if any(marker.startswith(('LSHO', 'RSHO', 'STRN')) for marker in missing_markers_list):
        if all(marker.startswith(('LSHO', 'RSHO', 'STRN')) for marker in missing_markers_list):
            print('All required markers are missing, returning NaN')
            return np.nan
        else:
            print('One or more required markers are missing, calculating trunk position with available markers')
            available_markers = []
            if not any(marker.startswith('LSHO') for marker in missing_markers_list):
                left_shoulder_rel = left_shoulder - left_shoulder[0]
                available_markers.append(left_shoulder_rel)
            if not any(marker.startswith('RSHO') for marker in missing_markers_list):
                right_shoulder_rel = right_shoulder - right_shoulder[0]
                available_markers.append(right_shoulder_rel)
            if not any(marker.startswith('STRN') for marker in missing_markers_list):
                sternum_rel = sternum - sternum[0]
                available_markers.append(sternum_rel)
            trunk = np.mean(available_markers, axis=0)
    else:
        # Normalize positions by subtracting initial positions
        left_shoulder_rel = left_shoulder - left_shoulder[0]
        right_shoulder_rel = right_shoulder - right_shoulder[0]
        sternum_rel = sternum - sternum[0]
        # Calculate the trunk position based on relative positions
        trunk = (left_shoulder_rel + right_shoulder_rel + sternum_rel)/ 3

    
    # Calculate the trunk velocity
    trunk_3Dvel = np.diff(trunk, axis=0) / np.diff(time)[:, np.newaxis] 
    
    # Calculate magnitude of velocity
    mag_vel = np.linalg.norm(trunk_3Dvel, axis=1)

    # Find the peak velocity
    peak_velocity = np.max(mag_vel)/1000
    
    return peak_velocity

peak_velocity_3d = calculate_3d_peak_velocity(LSHO,RSHO,STRN,time,missing_markers)
print("3D Peak velocity:", peak_velocity_3d,'m/s')




## Peak flexion angle of pelvis
Peak flexion angle of the pelvis, will be the min angle formed between the trunk-pelvis and trunk-knees vector.

The flexion angle is calculated as the angle between the two 3D vectors:
$$\alpha=\arccos \left(\frac{\boldsymbol{a} \cdot \boldsymbol{b}}{|\boldsymbol{a}||\boldsymbol{b}|}\right)$$

![hip_flexion.png](attachment:hip_flexion.png)
Img ref. Kellis et al., 2019 Effect of Hip Flexion Angle on the Hamstring to Quadriceps Strength Ratio


In [ ]:
def calculate_pelvis_flexion_angle(left_pelvis, right_pelvis, sternum, left_knee, right_knee,event_index,missing_markers_list):
    """
    Calculate the average angle of pelvis flexion based on the position of pelvic markers, sternum marker, and knee markers.

    Parameters:
    - left_pelvis (numpy.ndarray): Position vector representing the left pelvis marker.
    - right_pelvis (numpy.ndarray): Position vector representing the right pelvis marker.
    - sternum (numpy.ndarray): Position vector representing the sternum marker.
    - left_knee (numpy.ndarray): Position vector representing the left knee marker.
    - right_knee (numpy.ndarray): Position vector representing the right knee marker.

    Returns:
    - float: The average angle of pelvis flexion, in degrees.
    """
    if any(marker.startswith(('STRN', 'LASI', 'RASI', 'LKNE', 'RKNE')) for marker in missing_markers_list):
        if any(marker.startswith('STRN') for marker in missing_markers_list):
            print('Pelvis flexion: STRN marker is missing, returning NaN')
            return np.nan
        elif all(marker.startswith('LASI') for marker in missing_markers_list) or all(marker.startswith('LKNE') for marker in missing_markers_list):
            print('Pelvis flexion:Either LASI or LKNE markers are missing, using right pelvis and right knee markers')
            hip_sternum_vec = sternum - right_pelvis
            hip_knee_vec = right_knee - right_pelvis
        elif all(marker.startswith('RASI') for marker in missing_markers_list) or all(marker.startswith('RKNE') for marker in missing_markers_list):
            print('Pelvis flexion:Either RASI or RKNE markers are missing, using left pelvis and left knee markers')
            hip_sternum_vec = sternum - left_pelvis
            hip_knee_vec = left_knee - left_pelvis
        else:
            print('Pelvis flexion: Markers are missing, returning NaN')
            return np.nan

        dot_product = np.sum(hip_sternum_vec * hip_knee_vec, axis=1)
        norm = np.linalg.norm(hip_sternum_vec, axis=1) * np.linalg.norm(hip_knee_vec, axis=1)
        cos_angle = np.clip(dot_product / norm, -1.0, 1.0)
        angle = np.arccos(cos_angle)
        angle_degrees = np.degrees(angle)
        # angle_degrees = angle_degrees - angle_degrees[0]
        plt.plot(time,angle_degrees)
        return angle_degrees[event_index] #np.min(angle_degrees)#
    else:
        hip_sternum_vec1 = sternum - left_pelvis
        hip_sternum_vec2 = sternum - right_pelvis

        hip_knee_vec1 = left_knee - left_pelvis
        hip_knee_vec2 = right_knee - right_pelvis
        dot_product1 = np.sum(hip_sternum_vec1 * hip_knee_vec1, axis=1)
        dot_product2 = np.sum(hip_sternum_vec2 * hip_knee_vec2, axis=1)

        norm1 = np.linalg.norm(hip_sternum_vec1, axis=1) * np.linalg.norm(hip_knee_vec1, axis=1)
        norm2 = np.linalg.norm(hip_sternum_vec2, axis=1) * np.linalg.norm(hip_knee_vec2, axis=1)

        cos_angle1 = np.clip(dot_product1 / norm1, -1.0, 1.0)
        cos_angle2 = np.clip(dot_product2 / norm2, -1.0, 1.0)

        angle1 = np.arccos(cos_angle1)
        angle2 = np.arccos(cos_angle2)

        angle_degrees1 = np.degrees(angle1)
        angle_degrees2 = np.degrees(angle2)
        
        avg_angle = np.mean([angle_degrees1, angle_degrees2], axis=0)
        plt.plot(time,avg_angle)
        print(time[event_index])
        return avg_angle[event_index] #180 - np.min(avg_angle) #

# Example usage:
angle = calculate_pelvis_flexion_angle(LASI, RASI, STRN, LKNE, RKNE,global_min_index,missing_markers)
print("Max angle between hip-sternum line and hip-knees line:", angle)


## Mean Peak flexion angle of knees
Peak flexion angle of the knees, will be the min angle formed between the knee-pelvis and knee-heel vector. 

The flexion angle is calculated as the angle between the two 3D vectors:
$$\alpha=\arccos \left(\frac{\boldsymbol{a} \cdot \boldsymbol{b}}{|\boldsymbol{a}||\boldsymbol{b}|}\right)$$

The smaller the angle the higher the flexion


In [ ]:

def calculate_knee_flexion_angle(left_pelvis, right_pelvis, left_knee, right_knee,left_heel,right_heel,event_index,missing_markers_list):
    """
        Calculate the average angle of knee flexion based on the position of pelvic markers, knee markers, and heel markers.

        Parameters:
        - left_pelvis (numpy.ndarray): Position vector representing the left pelvis marker.
        - right_pelvis (numpy.ndarray): Position vector representing the right pelvis marker.
        - left_knee (numpy.ndarray): Position vector representing the left knee marker.
        - right_knee (numpy.ndarray): Position vector representing the right knee marker.
        - left_heel (numpy.ndarray): Position vector representing the left heel marker.
        - right_heel (numpy.ndarray): Position vector representing the right heel marker.

        Returns:
        - float: The average angle of knee flexion, in degrees.
        """
        
    if any(marker.startswith(('LASI', 'RASI', 'LKNE', 'RKNE', 'LHEE', 'RHEE')) for marker in missing_markers_list):
        if all(marker.startswith('LASI') for marker in missing_markers_list) and all(marker.startswith('RASI') for marker in missing_markers_list):
            print('Knee flexion:  LASI and RASI markers are missing, returning NaN')
            return np.nan
        elif all(marker.startswith('LKNE') for marker in missing_markers_list) and all(marker.startswith('RKNE') for marker in missing_markers_list):
            print('Knee flexion:  LKNE and RKNE markers are missing, returning NaN')
            return np.nan
        elif all(marker.startswith('LHEE') for marker in missing_markers_list) and all(marker.startswith('RHEE') for marker in missing_markers_list):
            print('Knee flexion:  LHEE and RHEE markers are missing, returning NaN')
            return np.nan
        elif any(marker.startswith('LASI') for marker in missing_markers_list) or any(marker.startswith('LKNE') for marker in missing_markers_list) or any(marker.startswith('LHEE') for marker in missing_markers_list):
            print('Knee flexion: Either LASI or LKNE or LHEE markers are missing, calculating with right side markers')
            hip_knee_vec = right_pelvis - right_knee
            knee_heel_vec = right_heel - right_knee
        elif any(marker.startswith('RASI') for marker in missing_markers_list) or any(marker.startswith('RKNE') for marker in missing_markers_list) or any(marker.startswith('RHEE') for marker in missing_markers_list):
            print('Knee flexion: Either RASI or RKNE or RHEE markers are missing, calculating with left side markers')
            hip_knee_vec = left_pelvis - left_knee
            knee_heel_vec = left_heel - left_knee
        else:
            return np.nan

        dot_product = np.sum(knee_heel_vec * hip_knee_vec, axis=1)
        norm = np.linalg.norm(knee_heel_vec, axis=1) * np.linalg.norm(hip_knee_vec, axis=1)
        cos_angle = np.clip(dot_product / norm, -1.0, 1.0)
        angle = np.arccos(cos_angle)
        angle_degrees = np.degrees(angle)
        plt.plot(angle_degrees)
        return angle_degrees[event_index] #np.min(angle_degrees)
    else:
        hip_knee_vec1 = left_pelvis - left_knee
        hip_knee_vec2 = right_pelvis - right_knee

        knee_heel_vec1 = left_heel - left_knee
        knee_heel_vec2 = right_heel - right_knee

        dot_product1 = np.sum(knee_heel_vec1 * hip_knee_vec1, axis=1)
        dot_product2 = np.sum(knee_heel_vec2 * hip_knee_vec2, axis=1)

        norm1 = np.linalg.norm(knee_heel_vec1, axis=1) * np.linalg.norm(hip_knee_vec1, axis=1)
        norm2 = np.linalg.norm(knee_heel_vec2, axis=1) * np.linalg.norm(hip_knee_vec2, axis=1)

        cos_angle1 = np.clip(dot_product1 / norm1, -1.0, 1.0)
        cos_angle2 = np.clip(dot_product2 / norm2, -1.0, 1.0)

        angle1 = np.arccos(cos_angle1)
        angle2 = np.arccos(cos_angle2)

        angle_degrees1 = np.degrees(angle1)
        angle_degrees2 = np.degrees(angle2)

        avg_angle = np.mean([angle_degrees1, angle_degrees2], axis=0)
        plt.plot(avg_angle)
        return avg_angle[event_index] #np.min(avg_angle) 

angle = calculate_knee_flexion_angle(LASI, RASI,LKNE, RKNE,LHEE,RHEE,global_min_index,missing_markers)


## Storing data

In [ ]:

#Create the dataframes to store the shoulder tilt and stroop angles
df_sit_chair = pd.DataFrame(columns=['Subject ID','Group','TrialFMA',
                                   'Peak_ML_Inclination[deg]','Peak_AP_Inclination[deg]',
                                       'Acc_RMS','Peak_trunk_vel[m/s]','Mean_trunk_jerk[m/s3]',
                                        'Max_pelvis_flexion[deg]','Max_knee_flexion[deg]'])

full_data = [fmap_data_control,fmap_data_pwp]
data_label = ['Control','PwP']

for i,group in enumerate(full_data):
    type_subject = data_label[i]
    for file in group:
                
       # Extract subject ID
        if type_subject == 'Control':
            subject_id = re.search(r'_C(\d+)\.tsv$', file).group(1)
            subject_id = 'C'+subject_id
        else: 
            subject_id = re.search(r'_P(\d+)\.tsv$', file).group(1)
            subject_id = 'P'+subject_id

        # Extract trial ID
        trial = re.search(r'_FMA0(\d+)_', file).group(1)

        flag,missing_markers,time,LFHD,RFHD,LBHD,RBHD,RSHO,LSHO,LSTC,RBAK,STRN,LASI,RASI,LPSI,RPSI,RTOE,LTOE,LANK,RANK,LHEE,RHEE,LKNE,RKNE = read_file(file)

        
        peak_ML,peak_AP, global_min_index = find_global_minima_and_inclination_angles(LSHO, RSHO, STRN,missing_markers)
        acc_rms = calculate_acc_rms(LSHO,RSHO,STRN,time,missing_markers)
        peak_trunk_velocity = calculate_3d_peak_velocity(LSHO,RSHO,STRN,time,missing_markers)
        mean_trunk_jerk = calculate_trunk_jerk(LSHO,RSHO,STRN,time,missing_markers)

        pelvis_flexion_angle = calculate_pelvis_flexion_angle(LASI, RASI, STRN, LKNE, RKNE,global_min_index,missing_markers)
        knee_flexion_angle = calculate_knee_flexion_angle(LASI, RASI,LKNE, RKNE,LHEE,RHEE,global_min_index,missing_markers)

        if knee_flexion_angle < 90:
            knee_flexion_angle = np.nan
        if pelvis_flexion_angle < 45:
            pelvis_flexion_angle = np.nan


        new_line = {'Subject ID':subject_id,'Group':type_subject,'TrialFMA':trial,
                   'Peak_ML_Inclination[deg]':peak_ML,'Peak_AP_Inclination[deg]':peak_AP,
                   'Acc_RMS':acc_rms,
                   'Peak_trunk_vel[m/s]':peak_trunk_velocity,
                    'Mean_trunk_jerk[m/s3]':mean_trunk_jerk,
                    'Max_pelvis_flexion[deg]':pelvis_flexion_angle,
                    'Max_knee_flexion[deg]':knee_flexion_angle}

            
        df_sit_chair.loc[len(df_sit_chair)] = new_line



In [ ]:
df_sit_chair

# Merge with covariates for ANCOVA